In [10]:
%load_ext autoreload
%autoreload 2

In [45]:
import sys
from vcdvcd import VCDVCD
import lcs, diff

def get_tv(fname, signame):
	vcd = VCDVCD(fname, signals=[signame], only_sigs=False)
	key = list(vcd.data.keys())[0]
	return vcd.data[key].tv

def split(tv):
	T, V = [], []
	for t, v in tv:
		T.append(t)
		V.append(v)
	return T, V

In [31]:
NAME = ["darkriscv-soc", "hazard3-soc"]
PREF = "/home/norbert/Projects/pico-ice/risc-v/cores/"
SUFF = "/tb/t12_custom.vcd"
file1 = PREF + NAME[0] + SUFF
file2 = PREF + NAME[1] + SUFF
sig1 = "t12_custom.soc.i_adr[31:0]"
sig2 = sig1

In [46]:
tv1 = get_tv(file1, sig1)
tv2 = get_tv(file2, sig2)

t1, v1 = split(tv1)
t2, v2 = split(tv2)

In [122]:
print(len(v1), len(v2))
print("lcs start")
LCS, LCS_v1i, LCS_v2i = lcs.lcs(v1, v2)
print("lcs stop", len(LCS))

33591 15514
lcs start
lcs stop 15227


In [ ]:
import pylcs
pylcs.lcs_sequence_length(str(v1), str(v2))

In [132]:
T = [t1, t2]
V = [v1, v2]
def cb(op, idx, s, e, i_lcs_common):
    lines = V[idx][s:e]
    if lines != ['100000000000111100']:
        # value, start, end common + common new
        vc,  tc1,  tc2  = LCS[i_lcs_common],   t1[LCS_v1i[i_lcs_common]]/1e6,   t2[LCS_v2i[i_lcs_common]]/1e6
        vcn, tc1n, tc2n = LCS[i_lcs_common+1], t1[LCS_v1i[i_lcs_common+1]]/1e6, t2[LCS_v2i[i_lcs_common+1]]/1e6
        vc  = hex(int(vc,  2))[2:]
        vcn = hex(int(vcn, 2))[2:]
        print(f"{NAME[idx]:20} {op}{len(lines):4} lines, last @{tc1:.0f}us/{tc2:.0f}us = {vc}, next @{tc1n:.0f}us/{tc2n:.0f}us = {vcn}")

diff.diff(v1, v2, LCS, cb)

darkriscv-soc        -   1 lines, last @37766us/20148us = 215c4, next @0us/4558us = 20000
hazard3-soc          +   1 lines, last @37766us/20148us = 215c4, next @0us/4558us = 20000
darkriscv-soc        -   1 lines, last @18396us/15844us = 2003c, next @18398us/15846us = 2015c
darkriscv-soc        -   1 lines, last @18420us/15868us = 201b4, next @18422us/15868us = 22518
darkriscv-soc        -   1 lines, last @18440us/15886us = 22560, next @18442us/15888us = 215dc
darkriscv-soc        -   1 lines, last @18480us/15918us = 21654, next @18508us/15920us = 21670
darkriscv-soc        -   1 lines, last @18512us/15922us = 21678, next @18514us/15922us = 21654
darkriscv-soc        -   1 lines, last @18520us/15928us = 21668, next @18522us/15928us = 215bc
darkriscv-soc        -   1 lines, last @18526us/15932us = 215c8, next @18528us/15932us = 2003c
darkriscv-soc        -   1 lines, last @18538us/15944us = 20064, next @18540us/15944us = 21668
darkriscv-soc        -   1 lines, last @18546us/15950us = 21

IndexError: list index out of range